# Label YouTube Comments for their Stance toward the U.S. Army

For this tutorial we are going to create labels for the [stance](https://www.sciencedirect.com/science/article/pii/S0306457322001728) of comments toward videos on the U.S. Army's official [YouTube Channel](https://www.youtube.com/USarmy). This type of labeling task is common for things like public affairs, political science, or marketing where we want yto get metrics on how certain messages are being received by the (a) public. 

In this context stance is defined as the opinion, either expressed or implied, of a user or text toward a target. Typically, stance is either labeled as 'for', 'against', 'neutral', and can include 'unrelated'.

In [ ]:
# install dependencies
! pip install -r requirements.txt

In [ ]:
# Import packages for labeling data by LLM
import pandas as pd  
import numpy as np
from tqdm import tqdm
tqdm.pandas()

from transformers import pipeline

from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_huggingface import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser

# Read in and inspect the dataset to be labeled

We will read in the validation dataset, which has human annotations to compare to, for this exercise. The full data set is available [here](https://zenodo.org/records/10493803)

In [ ]:
DATA_PATH = "@usarmy_comments_validation_set_labels.csv"

df = pd.read_csv(DATA_PATH, index_col=0)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

# Get an LLM working

For this exercise, we will stand up a local (relatively) small LLM, in this case a [specially tuned T5 model](https://huggingface.co/declare-lab/flan-alpaca-gpt4-xl). It should be noted that if you want to use a decoder-only model (i.e., Llama, Mistral, etc.) you need to switch to a `text-generation` pipeline. Also, setting `return_full_text=False` when using a text-generation pipeline is also helpful as it just returns what the model generates and not the full prompt.

Once we have the pipeline, we wrap it in langchain's pipeline class so that we can use it in chains.

Finally, one can also use a closed-source model, like OpenAI as well. Just consult [the documentation](https://python.langchain.com/docs/integrations/chat/openai/) to see how to do this

In [ ]:
# Load the model using Hugging Face pipeline
hf_pipeline = pipeline(
    "text2text-generation",
    model="declare-lab/flan-alpaca-gpt4-xl",
    device=-1,  # Use CPU (-1 for CPU, other numbers for GPUs)
    max_new_tokens = 100,
)

# Create the LangChain LLM using the HuggingFace pipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [ ]:
# run some examples 
question = '''Analyze the following social media post and determine its stance towards the provided entity. Respond with a single word: "for", "against", "neutral", or "unrelated". Only return the stance as a single word, and no other text.
entity: U.S. Army
post: @vondeveen If the Army wants to actually recruit people, maybe stop breaking people and actually prosecute sexual assualt #nomorewar.
stance:'''
print(llm.invoke(question))

In [ ]:
question = '''Analyze the following social media post and determine its stance towards the provided entity. Respond with a single word: "for", "against", "neutral", or "unrelated". Only return the stance as a single word, and no other text.
entity: U.S. Army
post: @artfulask I have never seen a pink-eared duck before. #Army
stance:'''
print(llm.invoke(question))

In [ ]:
question = '''Analyze the following social media post and determine its stance towards the provided entity. Respond with a single word: "for", "against", "neutral", or "unrelated". Only return the stance as a single word, and no other text.
entity: U.S. Army
post: I think the @Army helped me become disciplined. I would have surely flunked out of college chasing tail if I didn't get some discipline there. #SFL
stance:'''
print(llm.invoke(question))

# Create prompt templates

In [ ]:
context_template = '''Analyze the following YouTube comment to a video posted by the U.S. Army named "{title}" and determine its stance towards the provided entity. Respond with a single word: "for", "against", "neutral", or "unrelated". Only return the stance as a single word, and no other text.
        entity: {entity}    
        comment: {statement}    
        stance:'''  

# Initialize a PromptTemplate object  
context_prompt = PromptTemplate(input_variables=["title","entity","statement"], template=context_template) 

In [ ]:
example = df.iloc[0,:]

formated_prompt = context_prompt.format(title=example['name'], 
                      entity = "the U.S. Army",
                      statement = example['comment'])

print(formated_prompt)

# Create and Run a Labeling Chain

In the newer versions of LangChain, you string together 'runnbales' using the pipe (|) format to create chains

In [ ]:
llm_chain = context_prompt | llm |  StrOutputParser()

In [ ]:
llm_chain.invoke({"title":example['name'], 
                  "entity":"the U.S. Army",
                  "statement":example['comment']})

In [ ]:
# now, we can programmatically produce labels!

results = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Classifying rows"):
    result = llm_chain.invoke({
        "title": row['name'],
        "entity": "the U.S. Army",
        "statement": row['comment']
    })
    results.append(result)

In [ ]:
np.unique(results, return_counts=True)

As we can see in the output, sometimes we get extra text that we did not ask the LLM for. So, often we want a post-processing function to make sure everythign maps back to the labels we want

In [ ]:
def post_process_results(result):
    """
    This function post-processes the result from a large language model to label text.

    Args:
        result (str): A string representing the LLM output word.

    Returns:
        str: A classification label ('disagree', 'neutral', 'agree', or 'unrelated').
    """
    
    # Words or phrases that indicate each stance category
    disagree_indicators = ['against', 'denies', 'critical', 'deny', 'neg', 'oppose', 'opposes']
    agree_indicators = ['support', 'supports', 'for', 'pro ', 'positive', 'agree', 'agrees']
    neutral_indicators = ['neutral']

    # Normalize the word to lower case and remove leading/trailing white spaces
    normalized_word = str(result).strip().lower()

    # Determine stance based on the indicators
    if any(indicator in normalized_word for indicator in disagree_indicators):
        # If the word is also found in agree_indicators or neutral_indicators, label it as 'neutral'
        if any(indicator in normalized_word for indicator in agree_indicators) or any(indicator in normalized_word for indicator in neutral_indicators):
            return 'neutral'
        else:
            return 'against'
    elif any(indicator in normalized_word for indicator in neutral_indicators):
        # If the word is also found in disagree_indicators or agree_indicators, label it as 'neutral'
        if any(indicator in normalized_word for indicator in disagree_indicators) or any(indicator in normalized_word for indicator in agree_indicators):
            return 'neutral'
        else:
            return 'neutral'
    elif any(indicator in normalized_word for indicator in agree_indicators):
        # If the word is also found in disagree_indicators or neutral_indicators, label it as 'neutral'
        if any(indicator in normalized_word for indicator in disagree_indicators) or any(indicator in normalized_word for indicator in neutral_indicators):
            return 'neutral'
        else:
            return 'for'
    else:
        # If no specific stance label is found, label it as unrelated
        return 'unrelated'


In [ ]:
results = [post_process_results(i) for i in results]

In [ ]:
np.unique(results, return_counts=True)

# Prompt Engineering for Labeling Data by LLM

Okay, having seen how we can classify the stance of the comments toward a target (in this case, the U.S. Army), lets look at how we can construct some other labeling prompts, based on some of the design patterns we talked about earlier. Specifically, lets look at:
- few-shot prompting
- chain-of-thought-prompting

## Few-shot prompting

key to making this work well is the examples you give the LLM to reason on for classifying the stance. these examples coule be drawn from the same dataset, a related dataset or even completely made up.

In [ ]:
example_template = '''title: {title}
entity: {entity}
comment: {comment}
stance: {stance}'''

example_prompt = PromptTemplate(
    input_variables=["title", "entity", "comment", "stance"],
    template=example_template
)

examples = [
    {'title': "New Recruitment Video",
     'entity': "the U.S. Army",
     'comment': "This is an amazing initiative by the Army.",
     'stance': 'for'},
    
    {'title': "Training Highlights",
     'entity': "the U.S. Army",
     'comment': "This video shows the Army's commitment to readiness.",
     'stance': 'for'},
    
    {'title': "Military Expenditure Analysis",
     'entity': "the U.S. Army",
     'comment': "Why is so much taxpayer money wasted on this?",
     'stance': 'against'},
    
    {'title': "Veterans' Day Tribute",
     'entity': "the U.S. Army",
     'comment': "This is a neutral tribute, nothing special.",
     'stance': 'neutral'},
    
    {'title': "New Recruitment Video",
     'entity': "the U.S. Army",
     'comment': "This has nothing to do with the Army, totally irrelevant.",
     'stance': 'unrelated'},
]

In [ ]:
prefix = '''Stance classification is the task of determining the stance of a comment towards a specific entity. The following examples illustrate different stances a comment can take: "for", "against", "neutral", or "unrelated".'''

suffix = '''Analyze the following YouTube comment to a video posted by the U.S. Army named "{title}" and determine its stance towards the provided entity. Respond with a single word: "for", "against", "neutral", or "unrelated". Only return the stance as a single word, and no other text.
title: {title}
entity: {entity}
comment: {comment}
stance:'''

# Create the FewShotPromptTemplate using the updated prefix, suffix, and examples
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["title", "entity", "comment"],
    example_separator="\n\n"
)

In [ ]:
formated_prompt = few_shot_prompt.format(title=example['name'], 
                      entity = "the U.S. Army",
                      comment = example['comment'])

print(formated_prompt)

Now, we can simiarly define a chain for the few-shot prompting

In [ ]:
few_shot_chain = few_shot_prompt | llm |  StrOutputParser() | RunnableLambda(post_process_results)

In [ ]:
few_shot_chain.invoke({"title":example['name'], 
                  "entity":"the U.S. Army",
                  "comment":example['comment']})

## Chain-of-thought prompting

This method often requires constructing together multiple prompts, which breakdown and reason over the example to be classified.

In [ ]:
# CoT template 1: reason about potential stances

cot_template_1 = '''Analyze the following YouTube comment to a video named "{title}" posted by the U.S. Army. Consider the opinion, or stance, expressed in the comment about the provided entity. Provide reasoning for your analysis.
title: {title}
entity: {entity}
comment: {comment}
explanation:'''

cot_prompt_1 = PromptTemplate(
    input_variables=["title", "entity", "comment"],
    template=cot_template_1
)

cot_chain_1 = cot_prompt_1 | llm | StrOutputParser()


In [ ]:
cot_chain_1.invoke({"title":example['name'], 
                  "entity":"the U.S. Army",
                  "comment":example['comment']})

In [ ]:
# CoT template 1: prodcue the final stance judgement

cot_template_2 = '''Based on your explanation, "{stance_reason}", what is the final stance towards the provided entity? Respond with a single word: "for", "against", "neutral", or "unrelated". Only return the stance as a single word, and no other text.
title: {title}
entity: {entity}
comment: {comment}
stance:'''

cot_prompt_2 = PromptTemplate(
    input_variables=["title", "entity", "comment", "stance_reason"],
    template=cot_template_2
)

cot_chain_2 = cot_prompt_2 | llm | StrOutputParser()

In [ ]:
# Combine the chains together for labeling data points

cot_chain = {
    "stance_reason": cot_chain_1,
    "title": RunnablePassthrough(),
    "entity": RunnablePassthrough(),
    "comment": RunnablePassthrough()
} | cot_chain_2 | StrOutputParser() | RunnableLambda(post_process_results)

In [ ]:
cot_chain.invoke({"title":example['name'], 
                  "entity":"the U.S. Army",
                  "comment":example['comment']})